# **Import modules and data file**

In [5]:
from google.colab import files
files.upload()

{}

In [0]:
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard
from keras.layers import Dense, Dropout, Embedding, LSTM, TimeDistributed, Flatten
from keras.models import load_model, Sequential
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from pickle import dump, load
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import random

In [0]:
# import file
def load_file(filename):
    """
    - load file from filename and return a really long string
    - file is expected to be cleaned beforehand
    """
    file = open(filename, encoding = "utf8")
    raw_text = file.read()[:20000]
    file.close()
    return raw_text


def create_dict(raw_text):
    """
    create character dictionary and mapping to indices from data file
    """
    char = sorted(set(raw_text))
    mapping = {c : i for i, c in enumerate(char)}
    vocab_size = len(mapping)
    return mapping, vocab_size

def fixed_len_seq(raw_text, seq_len):
    """
    split data into sequences with fixed lengths
    """
    sequences = list()
    for i in range(seq_len, len(raw_text)):
        seq = raw_text[i-seq_len:i+1]
        sequences.append(seq)
    return sequences

def encode(sequences, encoder):
    """
    encode sequences text into integers
    """
    result = []
    for seq in sequences:
        encoded_seq = [encoder[char] for char in seq]
        result.append(encoded_seq)
    return result
        

def create_xy(sequences, vocab_size):
    # change here to adapt expected input shape for RNN layers
    """
    convert encoded sequences into predictor & label
    """
    sequences = np.array(sequences)
    X, y = sequences[:,:-1], sequences[:,-1]
    sequences = [to_categorical(x, num_classes = vocab_size) for x in X]
    X = np.array(sequences)
    y = to_categorical(y, num_classes = vocab_size)
    return X,y

        
def preprocess(filename, seq_len):
    """
    preprocess file intro appropriate format for Keras
    """
    raw_text = load_file(filename)
    sequences = fixed_len_seq(raw_text, seq_len)
    mapping, vocab_size = create_dict(raw_text)
    vocab_size = 71
    encoded_seq = encode(sequences, mapping)
    x, y = create_xy(encoded_seq, vocab_size)
    return x, y, mapping, vocab_size

# **Data preprocessing**

In [0]:
x_train, y_train, mapping, vocab_size = preprocess('khunchang_train.txt', 64)

In [0]:
x_val, y_val, mapping, vocab_size = preprocess('khunchang_val.txt', 64)

In [0]:
x_test, y_test, mapping, vocab_size = preprocess('khunchang_test.txt', 64)

In [191]:
x_train.shape
x_val.shape

(19936, 64, 71)

In [96]:
y_train.shape

(19936, 71)

# **Model training**

In [97]:
# model 1
# define model
batch_size = 64
drop_rate = 0.5
seq_len = 64
embedding_size = 32
rnn_size = 512
num_layers = 3
clip_norm = 5
learning_rate = 0.001

# build model
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_shape = (seq_len,)))
model.add(Dropout(drop_rate))
for _ in range(num_layers-1):
    model.add(LSTM(rnn_size, return_sequences=True))
    model.add(Dropout(drop_rate))
model.add(LSTM(rnn_size, return_sequences=False))
model.add(Dropout(drop_rate))
model.add(Dense(vocab_size, activation='softmax'))
#model.add(Flatten())
#model.add(TimeDistributed(Dense(vocab_size, activation="softmax")))
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_43 (Embedding)     (None, 64, 32)            2272      
_________________________________________________________________
dropout_113 (Dropout)        (None, 64, 32)            0         
_________________________________________________________________
lstm_91 (LSTM)               (None, 64, 512)           1116160   
_________________________________________________________________
dropout_114 (Dropout)        (None, 64, 512)           0         
_________________________________________________________________
lstm_92 (LSTM)               (None, 64, 512)           2099200   
_________________________________________________________________
dropout_115 (Dropout)        (None, 64, 512)           0         
_________________________________________________________________
lstm_93 (LSTM)               (None, 512)               2099200   
__________

In [98]:
# model 2
# compile model
model.compile(loss="categorical_crossentropy", optimizer=Adam(),
              metrics=['accuracy'])
# fit model
model.fit(x_train, y_train, epochs=100, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 335s - loss: 3.7421 - acc: 0.0602
Epoch 2/100
 - 321s - loss: 3.6845 - acc: 0.0680
Epoch 3/100
 - 206s - loss: 3.6773 - acc: 0.0704
Epoch 4/100
 - 207s - loss: 3.6950 - acc: 0.0666
Epoch 5/100
 - 167s - loss: 3.6693 - acc: 0.0704
Epoch 6/100
 - 167s - loss: 3.5626 - acc: 0.0852
Epoch 7/100
 - 166s - loss: 3.4456 - acc: 0.1050
Epoch 8/100
 - 166s - loss: 3.3663 - acc: 0.1183
Epoch 9/100
 - 167s - loss: 3.3070 - acc: 0.1306
Epoch 10/100
 - 169s - loss: 3.2115 - acc: 0.1484
Epoch 11/100
 - 168s - loss: 3.1035 - acc: 0.1696
Epoch 12/100
 - 165s - loss: 3.0248 - acc: 0.1844
Epoch 13/100
 - 165s - loss: 2.9620 - acc: 0.1925
Epoch 14/100
 - 166s - loss: 2.9403 - acc: 0.2042
Epoch 15/100
 - 167s - loss: 2.8448 - acc: 0.2227
Epoch 16/100
 - 165s - loss: 2.7810 - acc: 0.2358
Epoch 17/100
 - 166s - loss: 2.7229 - acc: 0.2509
Epoch 18/100
 - 164s - loss: 2.6565 - acc: 0.2717
Epoch 19/100
 - 165s - loss: 2.6011 - acc: 0.2795
Epoch 20/10

In [0]:
# save the model to file
model.save('model_2.h5')

# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

# **Generate text**

**model 2**

In [0]:
# load the model
model1 = load_model('model_1.h5')
model2 = load_model('model_2.h5')

# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

In [0]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		# encoded = to_categorical(encoded, num_classes=len(mapping))
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [219]:
a = generate_seq(model1, mapping, 64, '๏จะ', 500)
print(a)

๏จะคุค้ฟั้วกบุบหยายพันจะไป
ขุนไกรได้ฟังให้แม่เข้าไป	หัวนักนั้นแล้วไปมาถึงมา
ครั้นถึงลุกนั้นเป็นเห็นนาง	เป็นเล่นหมุมขึงจะคลอน
พลิ้งโร่งคร่อยว่องเมือนสงา	พระเดชรับหวานอุกใสรรรยา	พลังนั่นมีแล้วก็ไม่
ทิ่งตระบื่นนี่ควาดรรดคร	ดังไร
พระสวนั่นอกลามอุหมาบเพียงแต้	ปู่ย่าตายายสบายใจ	ข้าไกให้เกิดมาช้า	อักเฉิดเห็นเห็นผิดตระสายร่วงาย
แล้วก็กเที่นายส่านสองไม้มา
เลี้ยงมาก็เดือนสนร้องไป	ขุนช้างเอาเห็นเห็นผิดกลับถัวนาน
นางแต่ล้วหน้ายวลยองคลอนค์ผิงดนกลายดแล้วนิน
เที่ยวถึงเย่นไม่
อยู่ในแล้วก็ได้
เที้ยงมาก็สมรับพลาน
จะ


In [221]:
b = generate_seq(model2, mapping, 64, '๏จะ', 500)
print(b)

๏จะกล่าวถึงนางศรีประจัน	เป็นเศรษฐีมีพันธุ์ด้วยกันมา
อยู่ท่าพี่เลี้ยงเมืองสุพรรณ	ให้ห้าวันตามที่มีหมายมาฯ
๏ครานั้นพระองค์ผู้ทรงเดช	ทุกประเทศฦๅลบสยบสยอน
ครั้นจวนแจ้งแสงศรีรวีวร	พระภูธรเสด็จสู่ที่สรงชล
ไขสุหร่ายหยัดย้อยเป็นสองสายบ่าน	เอาเบี้ยบนลนลานเหน็บฝาเกลื่อน
บ้างเร่งหมอตำแยอย่าแชเชือน	ข่มท้องร้องเตือนลูกขวางตัว
บ้างก็เข้าหนุนหลังนั่งเคียงข้าง	กลายแก้วโดดแหวกเข้าแทรกกลาง	ชกหัวขุนช้างที่กลางเกลี้ยง
ขุนช้างทำหลับอยู่กับเตียง	ฝ่ายนางพิมนอนเคียงค่อยเมียงมอง
ขุนช้างวางร้องก้องกู่โวย	ขโมยลักเมียกูจู่จาก


# **Evaluate model on dev set**

In [209]:
model1.evaluate(x_val, y_val)

19936/19936 [==============================] - 25s 1ms/step


[5.542001793510841, 0.16873996789727128]

   32/19936 [..............................] - ETA: 30s

In [201]:
model2.evaluate(x_val, y_val)

19936/19936 [==============================] - 66s 3ms/step


[5.480580599311841, 0.17325441412520065]

# **Evaluate model on test set**

In [202]:
model1.evaluate(x_test, y_test)

19936/19936 [==============================] - 24s 1ms/step


[10.327602405609326, 0.020064205457463884]

In [206]:
model2.evaluate(x_test, y_test)

19936/19936 [==============================] - 66s 3ms/step


[9.633312054660118, 0.017054574638844303]